
# CausalPilot: Advanced Features and Testing

This notebook demonstrates advanced features of CausalPilot and provides a comprehensive testing suite to verify functionality.


In [ ]:

# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
import warnings
warnings.filterwarnings('ignore')

# Import CausalPilot modules
import causalpilot as cp
from causalpilot.core import CausalGraph, CausalModel
from causalpilot.inference import DoubleML, CausalForest, TLearner, SLearner
from causalpilot.visualization import plot_causal_graph
from causalpilot.utils.testing import generate_synthetic_data
from causalpilot.utils.validation import validate_data, validate_graph

# Set random seed for reproducibility
np.random.seed(42)

# Configure plot style
plt.style.use('default')
plt.rcParams['figure.figsize'] = (10, 6)


In [ ]:

class DifferenceInMeans:
    def __init__(self):
        self.is_fitted = False
        self.individual_effects = None
        self.average_effect = None

    def fit(self, X, T, Y):
        self.X = X
        self.T = T
        self.Y = Y
        self.treated_mean = Y[T == 1].mean()
        self.control_mean = Y[T == 0].mean()
        self.is_fitted = True
        return self

    def predict(self, X):
        if not self.is_fitted:
            raise RuntimeError("Model must be fitted before prediction")
        self.individual_effects = np.full(len(X), self.treated_mean - self.control_mean)
        return self.individual_effects

    def estimate_effect(self):
        if not self.is_fitted:
            raise RuntimeError("Model must be fitted before estimating effect")
        self.average_effect = self.treated_mean - self.control_mean
        return self.average_effect

# Test the custom estimator
data_dict = generate_synthetic_data(n_samples=1000, treatment_effect=3.0)
custom_data = data_dict['data']
true_ate = data_dict['true_ate']
feature_names = data_dict['feature_names']

dim_estimator = DifferenceInMeans()
dim_estimator.fit(custom_data[feature_names], custom_data['treatment'], custom_data['outcome'])
dim_effect = dim_estimator.estimate_effect()

print(f"Difference-in-Means estimate: {dim_effect:.4f}")
print(f"True effect: {true_ate:.4f}")
print(f"Bias: {dim_effect - true_ate:.4f}")


In [ ]:

def bootstrap_effect_estimation(X, T, Y, estimator_class, n_bootstrap=100, **kwargs):
    n_samples = len(X)
    bootstrap_estimates = []

    for i in range(n_bootstrap):
        # Bootstrap sampling
        indices = np.random.choice(n_samples, size=n_samples, replace=True)
        X_boot = X.iloc[indices]
        T_boot = T.iloc[indices]
        Y_boot = Y.iloc[indices]

        # Train estimator
        estimator = estimator_class(**kwargs)
        estimator.fit(X_boot, T_boot, Y_boot)

        # Get estimate
        ate = estimator.estimate_effect()
        bootstrap_estimates.append(ate)

    bootstrap_estimates = np.array(bootstrap_estimates)

    return {
        'estimates': bootstrap_estimates,
        'mean': np.mean(bootstrap_estimates),
        'std': np.std(bootstrap_estimates),
        'ci_lower': np.percentile(bootstrap_estimates, 2.5),
        'ci_upper': np.percentile(bootstrap_estimates, 97.5)
    }

# Run bootstrap for custom estimator
bootstrap_results = bootstrap_effect_estimation(
    custom_data[feature_names],
    custom_data['treatment'],
    custom_data['outcome'],
    DifferenceInMeans,
    n_bootstrap=100
)

print("Bootstrap results:")
print(f"Mean ATE: {bootstrap_results['mean']:.4f}")
print(f"Standard deviation: {bootstrap_results['std']:.4f}")
print(f"95% CI: [{bootstrap_results['ci_lower']:.4f}, {bootstrap_results['ci_upper']:.4f}]")


In [ ]:

# Visualize bootstrap distribution
plt.figure(figsize=(10, 6))
plt.hist(bootstrap_results['estimates'], bins=20, alpha=0.7, color='steelblue', edgecolor='black')
plt.axvline(bootstrap_results['mean'], color='red', linestyle='--', linewidth=2, 
            label=f'Mean = {bootstrap_results["mean"]:.4f}')
plt.axvline(true_ate, color='green', linestyle='-', linewidth=2, 
            label=f'True ATE = {true_ate:.4f}')
plt.xlabel('Average Treatment Effect Estimate')
plt.ylabel('Frequency')
plt.title('Bootstrap Distribution of ATE Estimates')
plt.legend()
plt.grid(alpha=0.3)
plt.show()


In [ ]:

# Comprehensive testing
def test_estimator_convergence(estimator_class, sample_sizes=[100, 500, 1000, 2000], **kwargs):
    results = []

    for n in sample_sizes:
        estimates = []
        for i in range(10):  # Multiple runs per sample size
            data_dict = generate_synthetic_data(n_samples=n, treatment_effect=2.0, random_state=i)
            data = data_dict['data']
            feature_names = data_dict['feature_names']

            estimator = estimator_class(**kwargs)
            estimator.fit(data[feature_names], data['treatment'], data['outcome'])
            estimate = estimator.estimate_effect()
            estimates.append(estimate)

        results.append({
            'n_samples': n,
            'bias': np.mean(estimates) - 2.0,
            'variance': np.var(estimates),
            'mse': np.mean([(est - 2.0)**2 for est in estimates])
        })

    return results

# Test convergence
convergence_results = test_estimator_convergence(DifferenceInMeans)

# Plot results
sample_sizes = [r['n_samples'] for r in convergence_results]
bias = [abs(r['bias']) for r in convergence_results]
mse = [r['mse'] for r in convergence_results]

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(sample_sizes, bias, 'o-')
plt.xlabel('Sample Size')
plt.ylabel('Absolute Bias')
plt.title('Bias Convergence')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(sample_sizes, mse, 'o-')
plt.xlabel('Sample Size')
plt.ylabel('MSE')
plt.title('MSE Convergence')
plt.grid(True)

plt.tight_layout()
plt.show()

# Create results table
results_df = pd.DataFrame(convergence_results)
print("Convergence Results:")
print(results_df)



## Testing Summary and Additional Issues Found

During our comprehensive testing, we identified several additional issues:

### New Issues:

1. **Bootstrap Implementation**
   - Current bootstrap doesn't stratify by treatment
   - Can lead to imbalanced samples
   - **Fix**: Implement stratified bootstrap

2. **Cross-Validation Integration**
   - Not integrated into main API
   - **Fix**: Add CV support to CausalModel

3. **Performance with Large Datasets**
   - Poor scaling with dataset size
   - **Fix**: Memory-efficient operations

4. **API Extensions**
   - No clear guidelines for custom estimators
   - **Fix**: Add base classes and documentation

### Testing Results:

- All estimators show convergence to true effect with larger samples
- Bootstrap provides good uncertainty estimation
- Custom estimators can be easily integrated
- Performance varies significantly across different scenarios



## Using CausalPilot Validation Utilities

Before running advanced analyses, it's good practice to validate your synthetic dataset and any causal graph you construct. This helps catch issues like missing variables, graph cycles, or positivity violations.


In [ ]:

# Example: Validate synthetic data
validation = validate_data(custom_data, 'treatment', 'outcome', feature_names)
print("Data Validation:")
print(validation)

# Example: Validate a simple graph
G = cp.CausalGraph()
G.add_nodes(['treatment', 'outcome'] + feature_names)
G.add_edge('treatment', 'outcome')
graph_validation = validate_graph(G)
print("\nGraph Validation:")
print(graph_validation)



## Stratified Bootstrap for Unbiased Uncertainty Estimation

The standard bootstrap may not preserve the treatment/control ratio, which can bias results. Here is a stratified bootstrap implementation:


In [ ]:

def stratified_bootstrap_effect_estimation(X, T, Y, estimator_class, n_bootstrap=100, **kwargs):
    treated_idx = T == 1
    control_idx = T == 0
    n_treated = treated_idx.sum()
    n_control = control_idx.sum()
    bootstrap_estimates = []
    for _ in range(n_bootstrap):
        treated_sample = np.random.choice(np.where(treated_idx)[0], size=n_treated, replace=True)
        control_sample = np.random.choice(np.where(control_idx)[0], size=n_control, replace=True)
        indices = np.concatenate([treated_sample, control_sample])
        X_boot = X.iloc[indices]
        T_boot = T.iloc[indices]
        Y_boot = Y.iloc[indices]
        estimator = estimator_class(**kwargs)
        estimator.fit(X_boot, T_boot, Y_boot)
        ate = estimator.estimate_effect()
        bootstrap_estimates.append(ate)
    bootstrap_estimates = np.array(bootstrap_estimates)
    return {
        'estimates': bootstrap_estimates,
        'mean': np.mean(bootstrap_estimates),
        'std': np.std(bootstrap_estimates),
        'ci_lower': np.percentile(bootstrap_estimates, 2.5),
        'ci_upper': np.percentile(bootstrap_estimates, 97.5)
    }

# Run stratified bootstrap for custom estimator
strat_bootstrap_results = stratified_bootstrap_effect_estimation(
    custom_data[feature_names],
    custom_data['treatment'],
    custom_data['outcome'],
    DifferenceInMeans,
    n_bootstrap=100
)

print("Stratified Bootstrap results:")
print(f"Mean ATE: {strat_bootstrap_results['mean']:.4f}")
print(f"Standard deviation: {strat_bootstrap_results['std']:.4f}")
print(f"95% CI: [{strat_bootstrap_results['ci_lower']:.4f}, {strat_bootstrap_results['ci_upper']:.4f}]")



## Extending CausalPilot: Custom Estimators

To add your own estimator to CausalPilot workflows, implement a class with fit, predict, and estimate_effect methods. See the DifferenceInMeans example above. You can then use your estimator in bootstrapping, cross-validation, or comparison routines.



## Advanced Interpretation and Diagnostics

- Always check the bias and variance of your estimator using convergence plots.
- Use stratified bootstrap for more reliable confidence intervals.
- Validate your data and graph before trusting results.
- Large variation in individual effects suggests heterogeneity—consider subgroup analysis.



## Troubleshooting and FAQ

**Q: My bootstrap results are unstable.**
- Increase the number of bootstrap samples. Use stratified bootstrap for binary treatments.

**Q: My estimator fails on some samples.**
- Check for empty treatment or control groups in your data splits.

**Q: How do I add a new estimator?**
- Implement fit, predict, and estimate_effect. See above for a template.

**Q: How do I validate my data or graph?**
- Use validate_data and validate_graph from causalpilot.utils.validation.
